<a href="https://colab.research.google.com/github/illNO/NLP_Igor_Shevchuk/blob/main/NLP_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import Bidirectional 
from tensorflow.keras.layers import Conv1D, MaxPooling1D 
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
from datasets import load_dataset
from tensorflow.keras.datasets import imdb
STOP_WORDS = ["went", "so", "the", "and", "a", "of", "from", "to", "not", "is", "in", "that", "this", "was", "as", "with", "for", "you", "are", "it"]

print(tf.__version__)

2.4.1


In [ ]:
# output directory name:
output_dir = 'model_output/LSTM/'

# training:
epochs = 4
batch_size = 128

# vector-space embedding: 
n_dim = 64 
max_review_length = 200 
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

# convolutional layer architecture:
n_conv = 64  
k_conv = 3 
mp_size = 4

# LSTM layer architecture:
n_lstm = 64 
drop_lstm = 0.2

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def tokenize_last(corpus):
  all_words = []
  for sent in corpus:
      tokenize_word = word_tokenize(sent)
      for word in tokenize_word:
          all_words.append(word)
  return all_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
all_words = tokenize_last(dataset['train']['text'])

In [ ]:
unique_words = set(all_words)
print(len(unique_words))

124836


In [ ]:
vocab_length = 5000

In [ ]:
from keras.preprocessing.text import one_hot

dataset = load_dataset("ag_news")
embedded_sentences_train = [one_hot(sent, vocab_length) for sent in dataset['train']['text']]
embedded_sentences_test = [one_hot(sent, vocab_length) for sent in dataset['test']['text']]

In [ ]:
x_train = pad_sequences(embedded_sentences_train, 200, padding='post')
x_valid = pad_sequences(embedded_sentences_test, 200, padding='post')

In [ ]:
y_train = np.array(dataset['train']['label'])
y_valid = np.array(dataset['test']['label'])

In [ ]:
n_dim = 200

model = Sequential()
model.add(Embedding(50000, n_dim, input_length=max_review_length)) 
model.add(SpatialDropout1D(drop_embed))
model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(MaxPooling1D(mp_size))
model.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm)))
model.add(Dense(4, activation='softmax'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 200, 200)          10000000  
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 200, 200)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 198, 64)           38464     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 49, 64)            0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128)               66048     
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 516       
Total params: 10,105,028
Trainable params: 10,105,028
Non-trainable params: 0
_________________________________________

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
x_valid.shape, y_valid.shape

((7600, 200), (7600,))

In [ ]:
x_train.shape, y_train.shape

((120000, 200), (120000,))

In [ ]:
# Batch size 128 returns error ( out of memory)
num_epochs = 4
model.fit(x_train, y_train, batch_size=16, epochs=num_epochs, validation_data=(x_valid, y_valid), verbose=1, callbacks=[modelcheckpoint])

Epoch 1/4
7500/7500 [==============================] - 535s 71ms/step - loss: 0.2821 - accuracy: 0.9033 - val_loss: 0.3114 - val_accuracy: 0.8950
Epoch 2/4
7500/7500 [==============================] - 533s 71ms/step - loss: 0.2381 - accuracy: 0.9181 - val_loss: 0.3019 - val_accuracy: 0.8974
Epoch 3/4
7500/7500 [==============================] - 530s 71ms/step - loss: 0.2062 - accuracy: 0.9286 - val_loss: 0.3047 - val_accuracy: 0.8989
Epoch 4/4
7500/7500 [==============================] - 530s 71ms/step - loss: 0.1821 - accuracy: 0.9368 - val_loss: 0.3038 - val_accuracy: 0.8991


In [ ]:
y_hat = model.predict_proba(x_valid)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [ ]:
"{:0.2f}".format(roc_auc_score(y_valid, y_hat)*100.0)